In [9]:
from onnx import TensorProto
from qonnx.core.modelwrapper import ModelWrapper
from slice_template import slice_node

tmp_model = ModelWrapper("tinyyolo_infershapes.onnx")

update_dict = {}
key_list = ["input", "starts", "ends", "axes", "splits", "output"]
drop_tensors = []



for s in tmp_model.get_nodes_by_op_type("Slice"):
    node_inx = tmp_model.get_node_index(s)
    vinfo = [tmp_model.get_tensor_valueinfo(inp) for inp in s.input]
    init = [tmp_model.get_initializer(inp) for inp in s.input]
    shapes = [tmp_model.get_tensor_shape(i) for i in s.input]
    t_names = [i for i in s.input]
    attr = {}
    for k in key_list:
        attr[k] = [None, None, None]
    for ind in range(len(shapes)):
        attr[key_list[ind]] = [t_names[ind], shapes[ind], init[ind]]
        drop_tensors.append(t_names[ind])

    for i in s.output:
        attr["output"] = [i, tmp_model.get_tensor_shape(i)]
        drop_tensors.append(i)
    print("*"*110)
    print(attr)
    x = slice_node(input_shape=attr["input"][1], output_shape=attr["output"][1],
                   starts_value=attr["starts"][2], ends_value=attr["ends"][2], axes_value=attr["axes"][2],
                   input_tensor=attr["input"][0], output_tensor=attr["output"][0],
                   dtype=TensorProto.INT64,node_name = s.name)
    update_dict[s.name] = [node_inx, x.make_node()]
    x.make_model(s.name+".onnx")


**************************************************************************************************************
{'input': ['735', [4], None], 'starts': ['737', [1], array([0])], 'ends': ['738', [1], array([2])], 'axes': ['736', [1], array([0])], 'splits': [None, None, None], 'output': ['739', [2]]}
**************************************************************************************************************
{'input': ['673', [], None], 'starts': ['675', [1], array([0])], 'ends': ['676', [1], array([2])], 'axes': ['674', [1], array([4])], 'splits': ['677', [1], array([1])], 'output': ['678', []]}
**************************************************************************************************************
{'input': ['673', [], None], 'starts': ['686', [1], array([2])], 'ends': ['687', [1], array([4])], 'axes': ['685', [1], array([4])], 'splits': ['688', [1], array([1])], 'output': ['689', []]}
***********************************************************************************************

In [37]:
import onnx
model = ModelWrapper("tinyyolo_infershapes.onnx")

replace_nodes = ["Slice_451"]

vinfo, keep_nodes, t_init, new_nodes = [], [], {}, []
tensors = model.get_all_tensor_names()

for t in tensors:
    if not (t in drop_tensors):
        vinfo.append(model.get_tensor_valueinfo(t))
        if not model.get_initializer(t) is None:
            t_init[t] = model.get_initializer(t)
replace_nodes = update_dict.keys()

for n in model.graph.node:
    if not n.name in replace_nodes:
        keep_nodes.append(n)

new_list = [v[1] for k,v in update_dict.items()]
for nnode in new_list:
    keep_nodes.append(nnode)
dropped_slice = ModelWrapper(onnx.helper.make_model(
    onnx.helper.make_graph(
        nodes= keep_nodes,
        name="tinyyolo_infershapes_updated",
        inputs=[model.graph.input[0]],
        outputs=[model.graph.output[0]],
        value_info=vinfo[0:-2]
        )
    )
)

for k,v in t_init.items():
    dropped_slice.set_initializer(k,v)
dropped_slice.save("Tiny_yolo_slice_update.onnx")
